### <span style='color:pink'>Codecademy jeopardy project -- data wrangling practice</span>

In [ ]:
# imports
import pandas as pd
import numpy as np
import os
import re

In [ ]:
# set path to data & load
input_folder = '/Users/alexandrabrown/Desktop/data_science/datasets/Jeopardy'
input_file   = 'jeopardy.csv'
input_path   = os.path.join(input_folder, input_file)

df = pd.read_csv(input_path)
df

### <span style='color:pink'>Update column names!</span>
remove leading spaces, replace middle spaces with '_', and make all text lowercase

In [ ]:
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()
df

### <span style='color:pink'>Write a function that filters the DataFrame for questions that contain all words in an input list</span>
The function below searches for words in a case insensitive manner since we make everything lowercase in the `lambda_filt` function

In [ ]:
def filter_by_question_words(df, word_list):
    # lambda function that filters for words in word_list and is case insensitive
    lambda_filt = lambda x: all(word.lower() in x.lower() for word in word_list)
    
    # apply lambda function to input DataFrame
    mask    = df['question'].apply(lambda_filt)
    # filter the DataFrame using the mask
    filt_df = df[mask].reset_index(drop=True)
    return filt_df
    
    
word_list = ['king', 'England']
filt_df = filter_by_question_words(df=df, word_list=word_list)
filt_df

### <span style='color:pink'>The way the question filtering function is writen above will also keep rows if the word of interest is within another word</span>
For example: if we filter by `['king', 'England']` then we also keep questions where king is part of a word: 'kingdom'
<span style='color:pink'>Update the question filtering function to only grab rows that have the words of interest -- not those where the word is part of another word</span>

In [ ]:
def refined_question_filter(df, word_list):
    
    # init list to store matching rows
    matching_rows = []
    
    # iterate over each row in the DataFrame
    for idx, row in df.iterrows():
        # grab the question for this row
        question = row['question']
        
        # turn question into list -- need to use regex here to prevent punctuation from being included in words
        question_list = re.findall(r"\b[\w']+\b", question.lower())
        
        # determine if there is a match
        if all(word.lower() in question_list for word in word_list):
            matching_rows.append(idx)
            
    return df.loc[matching_rows].reset_index(drop=True)
            
            
filt_df2 = refined_question_filter(df=df, word_list=word_list)
filt_df2

### <span style='color:pink'>More elegant version of the previous function!</span>
This time we define the funciton to operate on rows as opposed to operating on the entire DataFrame 

In [ ]:
word_list = ['king']

def row_matches(row, word_list):
    # grab question for this row
    question = row['question']
    
    # turn question text into list (tokenize)
    question_list = re.findall(r"\b[\w']+\b", question.lower())
    
    return all(word.lower() in question_list for word in word_list)

filt_df = df[df.apply(row_matches, word_list=word_list, axis=1)].reset_index(drop=True)
filt_df

### <span style='color:pink'>1) Convert value column from str to float 2) Use the new float column to find the difficulty of certain topics</span>
Example: what is the average value of questions that contain the word "King"?

In [ ]:
# clean up the value column & convert to float
df['value'] = (df['value']
               .astype(str)
               .str.strip()
               .str.replace(r'[$,]', '', regex=True))

df['value'] = pd.to_numeric(df['value'], errors='coerce')
df['value'] = df['value'].astype(float)


# avg val of questions with the word 'king'
word_list = ['king']
df_filt_by_word_val = df[df.apply(row_matches, word_list=word_list, axis=1)]
avg_word_val = df_filt_by_word_val['value'].mean()
print(f'The average value for questions containing the word "king" is: {avg_word_val:.2f}')

### <span style='color:pink'>Write function to find the number of unique answers to each question</span>
For example, after filtering the entire dataset for only questions containing the word "King", we could then find all of the unique answers to those questions. The answer “Henry VIII” appeared 45 times and was the most common answer

In [ ]:
def number_unique_answers(df, word_list):
    # filter the df by words in word list
    filt_df = df[df.apply(row_matches, word_list=word_list, axis=1)]
    # count number of instances for each answer in the filt_df -- save in dict
    answer_count_dict = filt_df['answer'].value_counts().to_dict()
    return answer_count_dict

# function call    
word_list = ['king']
answer_count_dict = number_unique_answers(df=df, word_list=word_list)

# display first key value pair from answer_count_dict
answer_iter = iter(answer_count_dict.items())
top_answer  = next(answer_iter)
print(top_answer)

### <span style='color:pink'>Next steps</span>
1) How many questions from the 90s use the word "Computer" compared to questions from the 2000s?

In [ ]:
# 1) Filter by year (grabbing 1990-1999) then use the row_matches func to find questions 
# with the word 'computer'

# convert dates from str to datetime
df['air_date'] = pd.to_datetime(df['air_date'])

# set start & stop dates for range of interest
start_date = '1990-01-01'
end_date   = '1999-12-31'

word = ['computer']

# filter by date range
df_90s = df[(df['air_date'] > start_date) & (df['air_date'] < end_date)]
# use date range filtered df and filter by word
df_90s_computer = df_90s[df_90s.apply(row_matches, word_list=word, axis=1)].reset_index(drop=True)
df_90s_computer


2. Is there a connection between the round and the category? Are you more likely to find certain categories, like "Literature" in Single Jeopardy or Double Jeopardy?

In [ ]:
# first pass exploration of the data
grouped_df = df.groupby('round')['category'].value_counts()


# we can make this easier to read using crosstab
cat_proportions = pd.crosstab(df['category'], df['round'], normalize='index')

# now filter for 'literature' not case sensitive!
cat_filt = cat_proportions[cat_proportions.index.str.contains('literature', case=False)]
# since there's a lot of different literature categories lets just sum the columns to see which 
# round has the most literature questions
cat_filt_sums = cat_filt.sum()
cat_filt_sums

3. Build a system to quiz yourself. Grab random questions, and use the input function to get a response from the user. Check to see if that response was right or wrong.

In [ ]:
# JEOPARD QUIZ FUNCTION
def jeopardy_quiz(df):
    # grab random sample question and answer from jeopardy df
    question_answer = df[['question', 'answer']].sample(n=1)
    
    # get question string
    question = question_answer['question'].iloc[0]
    # get answer string
    answer = question_answer['answer'].iloc[0]
    
    # ask user the question & prompt input
    user_input = input(question)
    
    # clean up answer and user input for comparison
    user_answer   = user_input.strip().lower()
    actual_answer = answer.strip().lower()
    
    # tokenize actual answer into words to prevent partial matches
    actual_answer_words = re.findall(r"\b[w']+\b", actual_answer)
    
    # compare user answer to actual answer words
    if user_answer in actual_answer_words:
        print('--> CORRECT!!')
    else:
        print(f'--> INCORRECT! {actual_answer} is the correct answer.')

### <span style='color:HotPink'>Run the next cell the play the jeopardy game!</span>
You will be prompted for an input!

In [ ]:
jeopardy_quiz(df=df)